In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pylamp.neural.layers import Linear
from pylamp.optim.optimizer import SGD
from pylamp.loss.losses import BCELoss
from pylamp.utils.data import DataGenerator as dg 

In [ ]:
X_train, X_test, y_train, y_test  = dg.generate_2D_data(data_type=0, epsilon=0.5)
dg.plot_2D_data(X_train, y_train)

In [ ]:
input_size = X_train.shape[1]
output_size = y_train.shape[1]
print(f'input_size: {input_size}, output_size: {output_size}')
model = Linear(input_size, output_size, use_bias=True)
loss = BCELoss()
train_loss, val_loss = SGD.step(
    model, loss, X_train=X_train, y_train=y_train, 
    epochs=100, lr=1e-3, batch_size=10, verbose=True,
)

In [ ]:
dg.plot_decision_boundary(X_test, y_test, model, "Boundary of Linear Data")